In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/sjsu/298B/298A/cogvideox_checkpoints"

test.pth  transformer_lora_epoch1.pth  transformer_lora_epoch2.pth  transformer_lora_epoch3.pth


In [ ]:
import torch
import torch.nn as nn
from diffusers import CogVideoXPipeline
from diffusers.utils import export_to_video
import os

from PIL import Image
from torchvision.utils import save_image
from torchvision.transforms import ToTensor
from tqdm import tqdm


import cv2
import numpy as np

prompt = "Crack two eggs into a bowl."

In [ ]:
# pipe = CogVideoXPipeline.from_pretrained(
#     "THUDM/CogVideoX-2b",
#     torch_dtype=torch.float16
# )

# pipe.enable_model_cpu_offload()
# pipe.enable_sequential_cpu_offload()
# # pipe.to("cuda")
# pipe.vae.enable_slicing()
# pipe.vae.enable_tiling()
# video = pipe(
#     prompt=prompt,
#     num_videos_per_prompt=1,
#     num_inference_steps=50,
#     num_frames=49,
#     guidance_scale=6,
#     generator=torch.manual_seed(42),
# ).frames[0]

# export_to_video(video, "output.mp4", fps=8)

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
class LoRALinear(nn.Module):
    def __init__(self, base, r=8, alpha=32):
        super().__init__()
        self.base = base
        self.lora_a = nn.Linear(base.in_features, r, bias=False)
        self.lora_b = nn.Linear(r, base.out_features, bias=False)
        self.scaling = alpha / r
        nn.init.kaiming_uniform_(self.lora_a.weight, a=5 ** 0.5)
        nn.init.zeros_(self.lora_b.weight)

    def forward(self, x):
        return self.base(x) + self.lora_b(self.lora_a(x)) * self.scaling

def inject_lora(transformer, keys=["to_q", "to_k", "to_v", "to_out.0"]):
    for name, module in transformer.named_modules():
        if any(k in name for k in keys):
            parts = name.split(".")
            parent = transformer
            for part in parts[:-1]:
                parent = getattr(parent, part)
            last = parts[-1]
            if isinstance(getattr(parent, last), nn.Linear):
                setattr(parent, last, LoRALinear(getattr(parent, last)))

# === Load base pipeline ===
pipe = CogVideoXPipeline.from_pretrained("THUDM/CogVideoX-2b", torch_dtype=torch.float16).to("cuda")
inject_lora(pipe.transformer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ckpt_path = "/content/drive/My Drive/sjsu/298B/298A/cogvideox_checkpoints/transformer_lora_epoch3.pth"
pipe.transformer.load_state_dict(torch.load(ckpt_path, map_location="cuda"))

pipe.transformer.to("cuda", dtype=torch.float16)
pipe.vae.to("cuda", dtype=torch.float16)
pipe.text_encoder.to("cuda", dtype=torch.float16)
# pipe.scheduler.to("cuda")  # if applicable

print("✅ Loaded fine-tuned weights!")

✅ Loaded fine-tuned weights!


In [ ]:
prompt = "Crack two eggs into a bowl."
# output = pipe(prompt=prompt, num_inference_steps=25, num_frames=8)
# video = output.frames[0]  # List of PIL.Image

video = pipe(
    prompt=prompt,
    num_videos_per_prompt=1,
    num_inference_steps=100,
    num_frames=49,
    guidance_scale=6,
    generator=torch.manual_seed(42),
).frames[0]


export_to_video(video, "lora_epoch3.mp4", fps=8)

# # Visualize
# import matplotlib.pyplot as plt
# fig, axs = plt.subplots(1, len(video), figsize=(20, 3))
# for i, frame in enumerate(video):
#     axs[i].imshow(frame)
#     axs[i].axis("off")
# plt.show()

  0%|          | 0/100 [00:00<?, ?it/s]

'lora_epoch3.mp4'

# prepare eval data

## gen video from model

### v1

In [ ]:
def save_video_with_gt_fps_and_jpgs(frames, save_path, gt_video_path, save_frames_dir=None):
    """
    Saves a list of PIL.Image frames into a video file,
    matching the FPS and resolution of the ground truth video.
    Optionally saves individual JPG frames.

    Args:
        frames (List[PIL.Image]): List of frames (PIL format).
        save_path (str): Output path to save video (.mp4).
        gt_video_path (str): Path to ground truth video (to extract FPS).
        save_frames_dir (str): Optional. If provided, will save each frame as JPG here.
    """
    assert len(frames) > 0, "No frames to save!"

    # === Get resolution from first frame ===
    width, height = frames[0].size

    # === Get FPS from ground truth video ===
    cap = cv2.VideoCapture(gt_video_path)
    if not cap.isOpened():
        raise ValueError(f"❌ Cannot open ground truth video: {gt_video_path}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    # === Define video writer ===
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    out = cv2.VideoWriter(save_path, fourcc, fps, (width, height))

    # === Optionally save frames to disk ===
    if save_frames_dir:
        os.makedirs(save_frames_dir, exist_ok=True)

    for i, frame in enumerate(frames):
        if isinstance(frame, Image.Image):
            np_frame = np.array(frame.convert("RGB"))
        else:
            np_frame = frame
        out.write(cv2.cvtColor(np_frame, cv2.COLOR_RGB2BGR))

        if save_frames_dir:
            frame_path = os.path.join(save_frames_dir, f"frame_{i+1:04d}.jpg")
            frame.save(frame_path)

    out.release()
    print(f"✅ Saved video to {save_path} with FPS={fps}")
    if save_frames_dir:
        print(f"🖼️  Saved frames to {save_frames_dir}")

In [ ]:
output_dir = "/content/drive/MyDrive/cogvideo_ft/cogvideo_eval"
os.makedirs(output_dir, exist_ok=True)

with open("/content/drive/MyDrive/cogvideo_ft/prompts.txt") as f:
    prompts = [line.strip() for line in f.readlines()]

with open("/content/drive/MyDrive/cogvideo_ft/videos.txt") as f:
    video_names = [line.strip().split("/")[-1] for line in f.readlines()]

for i, (prompt, gt_name) in enumerate(zip(prompts[:10], video_names[:10])):
    print(f"🌀 Generating for prompt: {prompt}")
    output = pipe(prompt=prompt, num_frames=49, num_inference_steps=25)
    video_frames = output.frames[0]  # List of PIL.Image

    save_path = os.path.join(output_dir, f"generated_{i}.mp4")
    gt_path = f"/content/drive/MyDrive/cogvideo_ft/videos/{gt_name}"
    save_frames_dir = os.path.join(output_dir, f"generated_frames")

    # save_video_with_gt_fps_and_jpgs(video_frames, save_path, gt_path, save_frames_dir)


    save_video_with_gt_fps_and_jpgs(
        video_frames,
        save_path,
        gt_path,
        save_frames_dir
    )

🌀 Generating for prompt: spread margarine on two slices of white bread


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Saved video to /content/generated_videos/generated_0.mp4 with FPS=30.0
🖼️  Saved frames to /content/generated_videos/generated_frames
🌀 Generating for prompt: place a slice of cheese on the bread


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Saved video to /content/generated_videos/generated_1.mp4 with FPS=30.0
🖼️  Saved frames to /content/generated_videos/generated_frames
🌀 Generating for prompt: place the bread slices on top of each other and place in a hot pan


...

✅ Saved video to /content/generated_videos/generated_7.mp4 with FPS=29.97
🖼️  Saved frames to /content/generated_videos/generated_frames
🌀 Generating for prompt: add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Saved video to /content/generated_videos/generated_8.mp4 with FPS=29.97
🖼️  Saved frames to /content/generated_videos/generated_frames
🌀 Generating for prompt: pour the dressing over the salad and mix


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Saved video to /content/generated_videos/generated_9.mp4 with FPS=29.97
🖼️  Saved frames to /content/generated_videos/generated_frames


In [ ]:
import shutil
import os

src_dir = "/content/generated_videos"
dst_dir = "/content/drive/MyDrive/cogvideo_ft/cogvideo_eval/generated_videos"

os.makedirs(dst_dir, exist_ok=True)

for filename in os.listdir(src_dir):
    if filename.endswith(".mp4"):
        shutil.copy(os.path.join(src_dir, filename), os.path.join(dst_dir, filename))

print("✅ Videos copied to Google Drive.")

✅ Videos copied to Google Drive.


In [ ]:
from pathlib import Path
import shutil

src_dir = Path("/content/generated_videos/generated_frames")
dst_dir = Path("/content/drive/MyDrive/cogvideo_ft/cogvideo_eval/generated_videos/generated_frames")
os.makedirs(dst_dir, exist_ok=True)

for filename in os.listdir(src_dir):
    if filename.endswith(".jpg"):
      shutil.copy(os.path.join(src_dir, filename), os.path.join(dst_dir, filename))

print("✅ JPGs copied to Google Drive.")

### v2

In [ ]:
def save_video_with_gt_fps_and_jpgs(frames, gt_video_path, save_root_dir):
    """
    Saves generated frames into a video and individual JPGs using the same basename as the ground truth video.

    Args:
        frames (List[PIL.Image]): List of frames (PIL format).
        gt_video_path (str): Ground truth video path to extract FPS and name.
        save_root_dir (str): Root directory to save output video and frames.
    """
    assert len(frames) > 0, "No frames to save!"
    basename = os.path.splitext(os.path.basename(gt_video_path))[0]  # e.g., '1k64hSAAAd4_clip0'

    # === Paths ===
    save_video_path = os.path.join(save_root_dir, "generated_videos", f"{basename}.mp4")
    save_frames_dir = os.path.join(save_root_dir, "generated_jpgs", basename)

    # === Create directories ===
    os.makedirs(os.path.dirname(save_video_path), exist_ok=True)
    os.makedirs(save_frames_dir, exist_ok=True)

    # === Get resolution and FPS ===
    width, height = frames[0].size
    cap = cv2.VideoCapture(gt_video_path)
    if not cap.isOpened():
        raise ValueError(f"❌ Cannot open GT video: {gt_video_path}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    # === Video writer ===
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(save_video_path, fourcc, fps, (width, height))

    for i, frame in enumerate(frames):
        if isinstance(frame, Image.Image):
            np_frame = np.array(frame.convert("RGB"))
        else:
            np_frame = frame
        out.write(cv2.cvtColor(np_frame, cv2.COLOR_RGB2BGR))

        frame_path = os.path.join(save_frames_dir, f"frame_{i+1:04d}.jpg")
        frame.save(frame_path)

    out.release()
    print(f"✅ Saved aligned video to: {save_video_path}")
    print(f"🖼️  Saved aligned frames to: {save_frames_dir}")


In [ ]:
output_dir = "/content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50"
os.makedirs(output_dir, exist_ok=True)

with open("/content/drive/MyDrive/cogvideo_ft/prompts.txt") as f:
    prompts = [line.strip() for line in f.readlines()]

with open("/content/drive/MyDrive/cogvideo_ft/videos.txt") as f:
    video_names = [line.strip().split("/")[-1] for line in f.readlines()]

for i, (prompt, gt_name) in enumerate(zip(prompts[:10], video_names[:10])):
    print(f"🌀 Generating for prompt: {prompt}")
    output = pipe(prompt=prompt, num_frames=49, num_inference_steps=50)
    video_frames = output.frames[0]  # List of PIL.Image

    # save_path = os.path.join(output_dir, f"generated_{i}.mp4")
    gt_path = f"/content/drive/MyDrive/cogvideo_ft/videos/{gt_name}"
    save_frames_dir = os.path.join(output_dir, f"generated_frames")

    # save_video_with_gt_fps_and_jpgs(video_frames, save_path, gt_path, save_frames_dir)


    # save_video_with_gt_fps_and_jpgs(
    #     video_frames,
    #     save_path,
    #     gt_path,
    #     save_frames_dir
    # )

    save_video_with_gt_fps_and_jpgs(video_frames, gt_path, output_dir)

🌀 Generating for prompt: spread margarine on two slices of white bread


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved aligned video to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_videos/GLd3aX16zBg_clip0.mp4
🖼️  Saved aligned frames to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_jpgs/GLd3aX16zBg_clip0
🌀 Generating for prompt: place a slice of cheese on the bread


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved aligned video to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_videos/GLd3aX16zBg_clip1.mp4
🖼️  Saved aligned frames to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_jpgs/GLd3aX16zBg_clip1
🌀 Generating for prompt: place the bread slices on top of each other and place in a hot pan


...

✅ Saved aligned video to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_videos/xHr8X2Wpmno_clip2.mp4
🖼️  Saved aligned frames to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_jpgs/xHr8X2Wpmno_clip2
🌀 Generating for prompt: add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved aligned video to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_videos/xHr8X2Wpmno_clip3.mp4
🖼️  Saved aligned frames to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_jpgs/xHr8X2Wpmno_clip3
🌀 Generating for prompt: pour the dressing over the salad and mix


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved aligned video to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_videos/xHr8X2Wpmno_clip4.mp4
🖼️  Saved aligned frames to: /content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_jpgs/xHr8X2Wpmno_clip4


## generate jpg frames from gt data

In [ ]:
def extract_frames_from_gt_videos(gt_video_dir, output_root_dir):
    os.makedirs(output_root_dir, exist_ok=True)

    for filename in tqdm(os.listdir(gt_video_dir)):
        if not filename.endswith(".mp4"):
            continue

        video_path = os.path.join(gt_video_dir, filename)
        video_name = os.path.splitext(filename)[0]
        output_dir = os.path.join(output_root_dir, video_name)
        os.makedirs(output_dir, exist_ok=True)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Failed to open {video_path}")
            continue

        frame_idx = 1
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_path = os.path.join(output_dir, f"frame_{frame_idx:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            frame_idx += 1

        cap.release()
        print(f"✅ Extracted {frame_idx - 1} frames for {video_name}")

In [ ]:


# === Run it ===
gt_video_dir = "/content/drive/MyDrive/cogvideo_ft/videos"
output_root_dir = "/content/drive/MyDrive/cogvideo_ft/gt_jpgs"

extract_frames_from_gt_videos(gt_video_dir, output_root_dir)

  0%|          | 1/300 [00:00<02:02,  2.45it/s]

✅ Extracted 49 frames for GLd3aX16zBg_clip0


  1%|          | 2/300 [00:00<01:52,  2.66it/s]

✅ Extracted 49 frames for GLd3aX16zBg_clip1


  1%|          | 3/300 [00:01<01:45,  2.82it/s]

...

✅ Extracted 49 frames for c9eELn4axpg_clip0


100%|█████████▉| 299/300 [01:54<00:00,  2.80it/s]

✅ Extracted 49 frames for c9eELn4axpg_clip1


100%|██████████| 300/300 [01:55<00:00,  2.61it/s]

✅ Extracted 49 frames for c9eELn4axpg_clip2


### gen jpg frames for modeled videos

In [ ]:
# === Run it ===
gt_video_dir = "/content/drive/MyDrive/cogvideo_ft/cogvideo_eval/generated_videos"
output_root_dir = "/content/drive/MyDrive/cogvideo_ft/cogvideo_eval/generated_jpgs"

extract_frames_from_gt_videos(gt_video_dir, output_root_dir)

  9%|▉         | 1/11 [00:00<00:02,  3.46it/s]

✅ Extracted 49 frames for generated_3


 18%|█▊        | 2/11 [00:00<00:02,  3.34it/s]

✅ Extracted 49 frames for generated_6


 27%|██▋       | 3/11 [00:00<00:02,  3.22it/s]

...

✅ Extracted 49 frames for generated_1


 82%|████████▏ | 9/11 [00:02<00:00,  3.16it/s]

✅ Extracted 49 frames for generated_7


100%|██████████| 11/11 [00:03<00:00,  3.44it/s]

✅ Extracted 49 frames for generated_9


# metrics

In [ ]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## v1

In [ ]:
import os
import glob
import torch
import lpips
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from torchvision import transforms

In [ ]:
# === Paths ===
gt_jpg_dir = "/content/drive/MyDrive/cogvideo_ft/gt_jpgs"
gen_jpg_dir = "/content/drive/MyDrive/cogvideo_ft/cogvideo_eval/generated_jpgs"

# === Setup ===
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])
lpips_fn = lpips.LPIPS(net='alex')

def load_image(path):
    return transform(Image.open(path).convert("RGB")).unsqueeze(0)

def calc_metrics(gt_folder, gen_folder):
    gt_paths = sorted(glob.glob(os.path.join(gt_folder, "frame_*.jpg")))
    gen_paths = sorted(glob.glob(os.path.join(gen_folder, "frame_*.jpg")))

    if len(gt_paths) != len(gen_paths):
        print(f"❌ Frame count mismatch: {len(gt_paths)} vs {len(gen_paths)}")
        return None

    ssim_total, psnr_total, lpips_total = 0, 0, 0
    for gt_p, gen_p in zip(gt_paths, gen_paths):
        gt_img = load_image(gt_p)
        gen_img = load_image(gen_p)

        gt_np = gt_img.squeeze().permute(1, 2, 0).numpy()
        gen_np = gen_img.squeeze().permute(1, 2, 0).numpy()

        ssim_val = ssim(gt_np, gen_np, channel_axis=2, data_range=1.0)
        psnr_val = psnr(gt_np, gen_np, data_range=1.0)
        lpips_val = lpips_fn(gt_img, gen_img).item()

        ssim_total += ssim_val
        psnr_total += psnr_val
        lpips_total += lpips_val

    n = len(gt_paths)
    return {
        "SSIM": ssim_total / n,
        "PSNR": psnr_total / n,
        "LPIPS": lpips_total / n
    }


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 153MB/s]


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


In [ ]:
# === Run for first 10 samples ===
results = []
for i in range(10):
    gt_name = os.path.splitext(os.listdir("/content/drive/MyDrive/cogvideo_ft/videos")[i])[0]
    gt_path = os.path.join(gt_jpg_dir, gt_name)
    gen_path = os.path.join(gen_jpg_dir, f"generated_{i}")

    if os.path.exists(gt_path) and os.path.exists(gen_path):
        metrics = calc_metrics(gt_path, gen_path)
        if metrics:
            results.append({"sample": gt_name, **metrics})

# === Show table ===
df = pd.DataFrame(results)
df

,sample,SSIM,PSNR,LPIPS
0,GLd3aX16zBg_clip0,0.337731,6.417313,0.777678
1,GLd3aX16zBg_clip1,0.371166,6.631341,0.839214
2,GLd3aX16zBg_clip2,0.152496,6.076470,0.736084
3,GLd3aX16zBg_clip3,0.381072,8.934414,0.670342
4,GLd3aX16zBg_clip4,0.339789,8.887317,0.707281
5,xHr8X2Wpmno_clip0,0.258513,7.664188,0.797911
6,xHr8X2Wpmno_clip1,0.454138,8.373821,0.572199
7,xHr8X2Wpmno_clip2,0.375904,8.708074,0.667186
8,xHr8X2Wpmno_clip3,0.232518,9.414665,0.717656
9,xHr8X2Wpmno_clip4,0.158187,10.276742,0.511281


## v2

In [ ]:
!pip uninstall -y clip

Found existing installation: clip 0.2.0
Uninstalling clip-0.2.0:
  Successfully uninstalled clip-0.2.0


In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-a3a8oj4d
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-a3a8oj4d
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=84ebcb0ca0b09977943a62de5ed51b20018385b301ac431fc3ff4dc62e2728b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-il9i8c3r/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
import lpips
import cv2
import clip

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
# === Metric setup ===
lpips_model = lpips.LPIPS(net="alex").cuda()
clip_model, clip_preprocess = clip.load("ViT-B/32", device="cuda")

# === Utility Functions ===

def load_image_tensor(path):
    img = Image.open(path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    return transform(img).unsqueeze(0).cuda()

def compute_ssim(img1, img2):
    img1 = np.array(img1.convert("L"))
    img2 = np.array(img2.convert("L"))
    return ssim(img1, img2)

def compute_psnr(img1, img2):
    img1 = np.array(img1)
    img2 = np.array(img2)
    return psnr(img1, img2)

def compute_lpips(img1_path, img2_path):
    img1 = lpips.im2tensor(lpips.load_image(img1_path)).cuda()
    img2 = lpips.im2tensor(lpips.load_image(img2_path)).cuda()
    return lpips_model(img1, img2).item()

def compute_clip_similarity(img_path, prompt):
    image = clip_preprocess(Image.open(img_path)).unsqueeze(0).cuda()
    text = clip.tokenize([prompt]).cuda()
    with torch.no_grad():
        image_features = clip_model.encode_image(image)
        text_features = clip_model.encode_text(text)
        similarity = torch.cosine_similarity(image_features, text_features).item()
    return similarity


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


100%|███████████████████████████████████████| 338M/338M [00:11<00:00, 29.7MiB/s]


In [ ]:
# === Main Evaluation Loop ===

gt_jpg_root = "/content/drive/MyDrive/cogvideo_ft/gt_jpgs"
# gen_jpg_root = "/content/drive/MyDrive/cogvideo_ft/cogvideo_eval/generated_jpgs"
gen_jpg_root = "/content/drive/MyDrive/cogvideo_ft/cogvideo_eval_epoch3_50/generated_jpgs"
prompt_file = "/content/drive/MyDrive/cogvideo_ft/prompts.txt"
video_list_file = "/content/drive/MyDrive/cogvideo_ft/videos.txt"

results = []

with open(prompt_file, "r") as pf, open(video_list_file, "r") as vf:
    prompts = [p.strip() for p in pf.readlines()]
    video_files = [v.strip().split("/")[-1] for v in vf.readlines()]  # e.g., 1k64hSAAAd4_clip0.mp4

    for i, video_name in enumerate(tqdm(video_files)):
        basename = video_name.replace(".mp4", "")
        gt_dir = os.path.join(gt_jpg_root, basename)
        # gen_dir = os.path.join(gen_jpg_root, f"generated_{i}")
        gen_dir = os.path.join(gen_jpg_root, basename)

        if not os.path.exists(gt_dir) or not os.path.exists(gen_dir):
            print(f"Skipping missing: {basename}")
            continue

        frame_names = sorted(os.listdir(gt_dir))
        ssim_vals, psnr_vals, lpips_vals = [], [], []

        for f in frame_names:
            gt_frame = os.path.join(gt_dir, f)
            gen_frame = os.path.join(gen_dir, f)
            if not os.path.exists(gt_frame) or not os.path.exists(gen_frame):
                continue

            img1 = Image.open(gt_frame).convert("RGB")
            img2 = Image.open(gen_frame).convert("RGB")

            ssim_vals.append(compute_ssim(img1, img2))
            psnr_vals.append(compute_psnr(img1, img2))
            lpips_vals.append(compute_lpips(gt_frame, gen_frame))

        clip_score = compute_clip_similarity(gen_frame, prompts[i])  # last frame and prompt

        results.append({
            "sample": basename,
            "SSIM": np.mean(ssim_vals),
            "PSNR": np.mean(psnr_vals),
            "LPIPS": np.mean(lpips_vals),
            "CLIP": clip_score
        })

# === Display Results ===
import pandas as pd
df = pd.DataFrame(results)
print(df)

# Optionally save:
df.to_csv("/content/drive/MyDrive/cogvideo_ft/cogvideo_eval/metrics_eval.csv", index=False)

100%|██████████| 300/300 [00:37<00:00,  7.99it/s]

Skipping missing: xHr8X2Wpmno_clip5
Skipping missing: V53XmPeyjIU_clip0
Skipping missing: V53XmPeyjIU_clip1
Skipping missing: V53XmPeyjIU_clip2
Skipping missing: V53XmPeyjIU_clip3
Skipping missing: V53XmPeyjIU_clip4
Skipping missing: V53XmPeyjIU_clip5
Skipping missing: V53XmPeyjIU_clip6
Skipping missing: V53XmPeyjIU_clip7
Skipping missing: mZwK0TBI1iY_clip0
Skipping missing: mZwK0TBI1iY_clip1
Skipping missing: mZwK0TBI1iY_clip2
Skipping missing: mZwK0TBI1iY_clip3
Skipping missing: mZwK0TBI1iY_clip4
Skipping missing: mZwK0TBI1iY_clip5
Skipping missing: mZwK0TBI1iY_clip6
Skipping missing: mZwK0TBI1iY_clip7
Skipping missing: H_9oM1Y60lU_clip0
Skipping missing: H_9oM1Y60lU_clip1
Skipping missing: H_9oM1Y60lU_clip2
Skipping missing: H_9oM1Y60lU_clip3
Skipping missing: H_9oM1Y60lU_clip4
Skipping missing: XOwypmUT5cc_clip0
Skipping missing: XOwypmUT5cc_clip1
Skipping missing: XOwypmUT5cc_clip2
Skipping missing: XOwypmUT5cc_clip3
Skipping missing: XOwypmUT5cc_clip4
Skipping missing: XOwypmUT5c

In [ ]:
df

,sample,SSIM,PSNR,LPIPS,CLIP
0,GLd3aX16zBg_clip0,0.423538,8.063868,0.826841,0.345947
1,GLd3aX16zBg_clip1,0.439133,7.894355,0.817969,0.332275
2,GLd3aX16zBg_clip2,0.285385,5.997174,0.847467,0.341309
3,GLd3aX16zBg_clip3,0.487052,9.033134,0.817116,0.278809
4,GLd3aX16zBg_clip4,0.371357,8.322691,0.839860,0.279541
5,xHr8X2Wpmno_clip0,0.387925,8.092943,0.976756,0.255371
6,xHr8X2Wpmno_clip1,0.449516,8.223316,0.851917,0.281738
7,xHr8X2Wpmno_clip2,0.508822,12.754574,0.808669,0.333984
8,xHr8X2Wpmno_clip3,0.349934,9.989094,0.785435,0.318115
9,xHr8X2Wpmno_clip4,0.278610,8.975298,0.747874,0.278564
